In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModel, AutoImageProcessor

In [15]:
model_name = "DAMO-NLP-SG/VideoLLaMA3-2B"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.

: 

In [6]:
import transformers
from typing import Union, List
import numpy as np, torch
from PIL import Image

# Define the alias if your version doesn't export it
if not hasattr(transformers.image_utils, "VideoInput"):
    transformers.image_utils.VideoInput = Union[
        List[Image.Image], np.ndarray, torch.Tensor, List[np.ndarray], List[torch.Tensor]
    ]


In [7]:
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00,  5.30it/s]


In [8]:
video_path = "input_10s.mp4"
question = "Describe this video in detail."

In [9]:
# Video conversation
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 128}},
            {"type": "text", "text": question},
        ]
    },
]

In [10]:
inputs = processor(conversation=conversation, return_tensors="pt")
inputs = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)

In [ ]:
output_ids = model.generate(**inputs, max_new_tokens=128)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.81 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 6.11 GiB is allocated by PyTorch, and 199.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(response)